In [26]:
import pandas as pd

df = pd.read_csv("../../tmp/proverbench-decomposed-batch0.csv")
# df = pd.read_csv("../../tmp/minif2f-decomposed.csv")

In [27]:
df

,benchmark,step,sanity,unigram\r\nsolved?,norm_num,linarith,nlinarith,omega,ring,ring_nf,simp,simpa,field_simp
0,aime_2024i_p2,step1.lean,1,1,0,1,1,0,0,0,0,1,0
1,aime_2024i_p2,step2.lean,1,1,0,1,1,0,0,0,0,0,0
2,aime_2024i_p2,step3.lean,1,1,0,1,1,0,0,0,0,0,0
3,aime_2024i_p2,step4.lean,1,1,0,1,1,0,0,0,0,1,0
4,aime_2024i_p2,step5.lean,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,aime_2025ii_p13,step2.lean,1,0,0,0,0,0,0,0,0,0,0
259,aime_2025ii_p15,step1.lean,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
260,aime_2025ii_p15,step2.lean,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
261,aime_2025ii_p15,step3.lean,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [28]:
df_sliced = df.iloc[:, 4:].copy()
df_sliced[df_sliced != 1] = 0 # set all others to 0
df_sliced = df_sliced[(df_sliced != 0).any(axis=1)]
df_sliced

,norm_num,linarith,nlinarith,omega,ring,ring_nf,simp,simpa,field_simp
0,0,1,1,0,0,0,0,1,0
1,0,1,1,0,0,0,0,0,0
2,0,1,1,0,0,0,0,0,0
3,0,1,1,0,0,0,0,1,0
5,0,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
222,0,0,0,1,0,0,0,1,0
224,0,0,0,1,0,0,0,1,0
225,0,0,0,1,0,0,0,1,0
230,0,1,1,0,0,0,0,0,0


In [29]:
# do a simple counting

from tabulate import tabulate
from collections import Counter

unigrams = list(df_sliced.columns)

tuples = [tuple(row) for row in df_sliced.values]
freq_counter = Counter(tuples)

table = []

for tup, count in freq_counter.most_common():
    pat = ", ".join([unigrams[i] for i in range(len(tup)) if tup[i] == 1])
    # if pat.strip() == "":
    #     continue
    table.append((count, pat))
    # print(f"{pat}: {count}")

tb = tabulate(table, headers=["count", "pattern"], tablefmt="github")
print(tb)

|   count | pattern                                                       |
|---------|---------------------------------------------------------------|
|      30 | linarith, nlinarith                                           |
|      23 | linarith, nlinarith, simpa                                    |
|      12 | omega, simpa                                                  |
|       9 | norm_num, simp, simpa, field_simp                             |
|       8 | nlinarith                                                     |
|       7 | norm_num, omega, simp, simpa, field_simp                      |
|       6 | simpa                                                         |
|       6 | linarith, nlinarith, omega, simpa                             |
|       5 | linarith, nlinarith, omega                                    |
|       4 | norm_num, linarith, nlinarith, omega, simp, simpa, field_simp |
|       2 | norm_num, ring, ring_nf, simp, simpa, field_simp              |
|       1 | 

In [32]:
# do an association rule analysis

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = apriori(df_sliced, min_support=0.1, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=1.0)

/Users/joseph/anaconda3/envs/py312/lib/python3.12/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [33]:
rules_deterministic = rules[rules['confidence'] == 1.0]
rules_deterministic = rules_deterministic.sort_values(by="lift", ascending=False)
print(rules_deterministic[["antecedents", "consequents"]].to_string(index=False))

                  antecedents                   consequents
                       (simp)                    (norm_num)
                   (norm_num)            (field_simp, simp)
                       (simp) (field_simp, norm_num, simpa)
            (norm_num, simpa)            (field_simp, simp)
                (simp, simpa)        (field_simp, norm_num)
             (simp, norm_num)           (field_simp, simpa)
          (field_simp, simpa)              (simp, norm_num)
       (field_simp, norm_num)                 (simp, simpa)
           (field_simp, simp)             (norm_num, simpa)
(field_simp, norm_num, simpa)                        (simp)
    (field_simp, simp, simpa)                    (norm_num)
                       (simp)           (field_simp, simpa)
          (field_simp, simpa)                        (simp)
                   (norm_num)                        (simp)
          (field_simp, simpa)                    (norm_num)
                   (norm_num)           